In [1]:
import pandas as pd
import re

In [2]:
#read in the csv files into df
df18 = pd.read_csv('/Users/dillonquan/Desktop/ds_jobs/alldata.csv')
df19 = pd.read_csv('indeed2019All.csv', encoding = 'latin-1')

In [3]:
#run this to make sure there are no duplicates within the data when sampling
df = df19.groupby(['position', 'company','location']).sum().reset_index().sort_values(by = 'flag', ascending = False)
df19 = ''
#create a state column
df['state'] = df['location'].str[-2:]

In [4]:
def categorize(s):
    patterns = ['.*data sci.*','.*data.*eng.*', '.*analy.*', '.*machine.*']
    for idx, pattern in enumerate(patterns):
        reg = re.compile(pattern)
        if reg.search(s):
            return idx
    return -1

In [5]:
#random sample 7000 job postings and setting the big dataframe to be nothing
df_ran = df.sample(n = 7000, random_state = 1)

In [6]:
df_ran['position'] = df_ran['position'].apply(lambda x : x.lower())
df_ran = df_ran.reset_index(drop = True)
df_ran.head()

,position,company,location,flag,state
0,quantitative researcher,MKT MEDIASTATS,"Cambridge, MA",5,MA
1,distinguished data scientist,Verizon,"Irving, TX",5,TX
2,senior data analytics analyst,Allstate,"Chicago, IL",1,IL
3,senior organizational change consultant,itelligence,"Chicago, IL",1,IL
4,adobe analytics analyst,Turbine Labs,"Greenwood Village, CO",1,CO


In [7]:
df_ran['label'] = -1
for idx, row in df_ran.iterrows():
    cat = categorize(row.position)
    if cat != -1:
        df_ran.loc[idx, 'label'] = cat

In [8]:
d19 = df_ran.groupby('label').count().reset_index()

In [9]:
df18['position'] = df18['position'].apply(lambda x : str(x).lower())
df18 = df18.reset_index(drop = True)
df18.head()

,position,company,description,reviews,location
0,development director,ALS TDI,Development Director\nALS Therapy Development ...,NaN,"Atlanta, GA 30301"
1,an ostentatiously-excitable principal research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",NaN,"Atlanta, GA"
2,data scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",NaN,"Atlanta, GA"
3,data analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,44.0,"Atlanta, GA 30303"
4,assistant professor -tt - signal processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,550.0,"Atlanta, GA"


In [10]:
df18['label'] = -1
for idx, row in df18.iterrows():
    cat = categorize(row.position)
    if cat != -1:
        df18.loc[idx, 'label'] = cat

In [11]:
d18 = df18.groupby('label').count().reset_index()

In [12]:
d18

,label,position,company,description,reviews,location
0,-1,3759,3748,3748,2921,3748
1,0,1444,1444,1444,1035,1444
2,1,218,218,218,157,218
3,2,1079,1079,1079,875,1079
4,3,464,464,464,338,464


In [13]:
t1 = d19[['label', 'position']].loc[(d19.label >= 0),:]
t2 = d18[['label', 'position']].loc[(d18.label >= 0),:]

In [17]:
t1.to_csv('2019_count.csv')

In [18]:
t2.to_csv('2018_count.csv')

In [19]:
ls

2018_count.csv
2019_EDA_Indeed.ipynb
2019_count.csv
Data Scientist Job Search Terms - general skills.csv
Data Scientist Job Search Terms - software skills.csv
Exploration.ipynb
README.md
Statebins.ipynb
Web_Scraping_Indeed.ipynb
indeed2019All.csv
indeed2019_rest.csv
indeed_2019.csv


In [16]:
df = ''

In [ ]:
%load_ext rpy2.ipython
%R library(ggplot2)
%R library(scales)

In [ ]:
%R -i t1
%R i t2

for d in [d19, d18]:
    for idx, row in d.iterrows():
        row

In [10]:
df.groupby('state').count().sort_values(by='flag', ascending = False)

,position,company,location,flag
state,,,,
CA,4152,4152,4152,4152
NY,2949,2949,2949,2949
MA,2751,2751,2751,2751
GA,2440,2440,2440,2440
NJ,2322,2322,2322,2322
WA,1942,1942,1942,1942
TX,1791,1791,1791,1791
IL,1771,1771,1771,1771
CO,1521,1521,1521,1521


## Did not run cells down here

In [ ]:
#include all the other states that we dont have information on
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
"HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
"MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
"NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
"SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

states = pd.DataFrame(states)
states.columns = ['states']
state = pd.merge(state, states, left_on = 'state', right_on ='states', how = 'right')
state = state.fillna(0)

In [ ]:
state = state.drop('state', axis = 1).rename(columns={'states':'state'})
state['company'] = state['company'].astype(int)
state['company'] = state['company']*-1
state.head()

In [ ]:
%%R

statebins(state, value_col = "company") +
labs(title = 'Number of Data Science Jobs per State 2018') +
scale_fill_distiller(palette = "Blues") +
theme_statebins(legend_position = "none")